# HBA PA5

French version: https://github.com/Daniel-R-Armstrong/French-Wiki-2500-Pretrained-SentencePiece-LM/blob/master/fr_spm.ipynb
https://forums.fast.ai/t/sentencepiece/53010/50

# Turkish ULMFiT from scratch

In [1]:
import warnings; warnings.simplefilter('ignore')

## HBA: TextList is in fastaiv1
https://fastai1.fast.ai/text.data.html#TextList.from_folder
https://docs.fast.ai/text.data#TextBlock.from_folder

In [2]:
import sys
!{sys.executable} -m pip install fastai==1.0.61 #This right here

     |████████████████████████████████| 239 kB 12.2 MB/s eta 0:00:01
  Attempting uninstall: fastai
    Found existing installation: fastai 2.1.8
    Uninstalling fastai-2.1.8:
      Successfully uninstalled fastai-2.1.8
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

fastbook 0.0.11 requires fastai>=2.0, but you'll have fastai 1.0.61 which is incompatible.


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *

Matplotlib is building the font cache; this may take a moment.


In [4]:
from fastai.imports import *

In [5]:
data_path = Path('/home/ec2-user/.fastai/data')

In [6]:
bs=128
# torch.cuda.set_device(0)
# data_path = Config.data_path()
# data_path = '/home/ec2-user/.fastai/data'

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [7]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

In [18]:
lm_fns

[PosixPath('/home/ec2-user/.fastai/data/trwiki/models/tr_wt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/models/tr_wt_vocab')]

In [8]:
path

PosixPath('/home/ec2-user/.fastai/data/trwiki')

In [9]:
lang

'tr'

## Turkish wikipedia model

In [26]:
from nlputils import split_wiki,get_wiki

## Uncompress wiki file from cli

Source: https://github.com/attardi/wikiextractor

1. Do `python setup.py install`
1. Double check with: `which wikiextractor`

`
base) [ec2-user@ip-172-16-69-8 trwiki]$ which wikiextractor
~/anaconda3/bin/wikiextractor
`

1. Uncrompress

`
(base) [ec2-user@ip-172-16-69-8 trwiki]$ python -m wikiextractor.WikiExtractor trwiki-latest-pages-articles.xml
INFO: 2832129   Mucize Dünyası: New York – Birleşik Kahramanlar
INFO: 2832111   Kraliyet Mühendislik Akademisi üyesi kadınlar listesi
INFO: 2832131   Ilana Rovina
INFO: 2832155   Artur Taymazov
INFO: 2832130   Enka (kadın voleybol takımı) 1992-93 sezonu
INFO: Finished 31-process extraction of 377951 articles in 123.1s (3071.5 art/s)
INFO: total of page: 732729, total of articl page: 377951; total of used articl page: 377951
(base) [ec2-user@ip-172-16-69-8 trwiki]$ which wikiextractor
~/anaconda3/bin/wikiextractor
`
    


In [10]:
get_wiki(path,lang)

/home/ec2-user/.fastai/data/trwiki/trwiki already exists; not downloading


In [11]:
# from nlputils import split_wiki,get_wiki

# get_wiki(path,lang)
!head -n4 {path}/{name}

<doc id="10" url="https://tr.wikipedia.org/wiki?curid=10" title="Cengiz Han">
Cengiz Han

Cengiz Han (doğum Temuçin, – 18 Ağustos 1227) ölümünden sonra tarihin en büyük yüzölçümüne sahip imparatorluğu haline gelmiş Moğol İmparatorluğu'nun kurucusu ve ilk Kağanı ve İmparatoru olan Moğol komutan ve hükümdar. 13. Yüzyılın başında Orta Asya'daki tüm göçebe bozkır kavimlerini birleştirerek bir ulus hâline getirdi ve o ulusu "Moğol" siyasi kimliği çatısı altında topladı.


In [12]:
path.ls()

[PosixPath('/home/ec2-user/.fastai/data/trwiki/trwiki-latest-pages-articles.xml.bz2'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/wikiextractor-orig'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trwiki-latest-pages-articles.xml'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/log'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/wikiextractor'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trwiki'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/models')]

In [27]:
dest = split_wiki(path,lang)

/home/ec2-user/.fastai/data/trwiki/docs already exists; not splitting


In [28]:
dest.ls()[:5]

[PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Kara delik.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Heaviside katmanı.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Zurna.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Özallı Yıllar.txt'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/docs/Kuşça, Cihanbeyli.txt')]

In [29]:
dest

PosixPath('/home/ec2-user/.fastai/data/trwiki/docs')

Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

![Turkish morphemes example](images/turkish.jpg)

### File Locations language model, vocab, databunch

In [ ]:
/home/ec2-user/SageMaker/fast-nlp/course-nlp/dotfastai-data/trwiki/tmp/spm.model
/home/ec2-user/SageMaker/fast-nlp/course-nlp/dotfastai-data/trwiki/tmp/spm.vocab
/home/ec2-user/SageMaker/fast-nlp/course-nlp/dotfastai-data/trwiki/docs/tr_databunch

## For output of this Training job. See Tweet: 

- https://twitter.com/HBAkirmak/status/1322256689022775297
- https://twitter.com/HBAkirmak/status/1322148506069512193?s=20

# From NN-IMDB NB

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [11]:
path

PosixPath('/home/ec2-user/.fastai/data/trwiki')

In [12]:
path_clas = path/'trmoviesentiment'
path_clas.ls()

[PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/tr_clas_databunch'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/tr_polarity.pos'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/Turkish_Movie_Sentiment.zip'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/tr_polarity.neg'),
 PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment/models')]

In [13]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [14]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [15]:
df = pd.concat([pos_df,neg_df], sort=False)

In [16]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [17]:
data_lm.show_batch()

idx,text
0,"sa ▁bile ▁sinema ▁olgusu nun ▁en ▁üst ▁noktalar in dan . . ▁xxbos ▁gerçekten ▁tarihi ▁sav as ▁filmleri ▁ara si nda ▁tar tis ma siz ▁en ▁iyi si ▁ , ▁12 ▁ yi l ▁boyunca ▁ac aba ▁ikincisi ▁çek ir imi ▁diye ▁bekledi gim ▁bir ▁film ▁ , bel ki ▁william ▁wallace ▁baba sinin ▁ölümünden ▁sonra ▁amca si ▁yani na ▁al mis ti ▁onu ▁ ye tis tir mis ti"
1,le ni siyle ▁insana ▁ iste ▁film ▁böyle ▁çekilir ▁de dir ten ▁kusur suz ▁bir ▁film . ▁xxbos ▁böyle ▁güzel ▁bir ▁yap it ▁olamaz ▁filmde ▁her ▁sey ▁var ▁insani ▁dünya dan ▁ali p ▁götür üyor ▁bask a ▁diya r lara ▁film ▁bitti kten ▁sonra ▁epey ▁süre ▁geçmesi ▁gerekiyor ▁tekrar ▁dünya ▁ya ▁dönmek ▁için ▁dikkat ! . ▁xxbos ▁ in an ir mi siniz ▁bu ▁filmi ▁ ne ▁kadar ▁izledi gi
2,▁wallace  in ▁i sk ence ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁ unu ta mi yorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinema da ▁izledi m ▁ve ▁insan in ▁ in an di ktan ▁sonra ▁ ne leri ▁yap a bile ce gi ni ▁fark ▁etti m . ▁gerçekten ▁süper ▁film di . ▁halen ▁içi
3,▁uzun lug una ▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart mi yorum ▁hayat im da ▁izledi gim ▁en ▁iyi ▁film lerden ▁biri ▁diye bilir im . tam ▁bir ▁bas yap it ▁nitel ig inde . o scar ▁al di gina ▁hiç ▁
4,▁filmlerinden ▁biri . ▁diyalog lar ▁çok ▁iyi . ▁kesinlikle ▁izlenmesi ▁gereken ▁bir ▁film . . ▁xxbos ▁tarantino nun ▁bu ▁filmi ▁kendini ▁belli ▁etti r iyor . hat ta ▁ben ce ▁tarantino nun ▁en ▁iyi ▁filmidir . kendi ne ▁has ▁ anlat imi ▁ile ▁bu ▁film ▁hak ka ten ▁sinema ▁sever lerin ▁izlemesi ▁gereken ▁bir ▁film . ben ▁10 ▁üzerinden ▁7 ▁verdi m ▁bu ▁benim ▁için ▁oldukça ▁çok ▁puan . ▁xxbos


# s3 backup HBA
s3://<folder>-ml/trlanguagemodel/models/

`
(base) $ aws s3 ls s3://<folder>-ml/trlanguagemodel/models/
2020-10-31 21:16:33  387655145 learner_mod_tr_spm_save.pkl.pth
2020-10-31 21:16:33  131193699 learner_modtr_spm.pkl
2020-10-31 21:16:33  129098532 learner_tr_spm_enc.pth
2020-10-31 21:16:33     577263 learner_vocab_tr_spm.pkl
2020-10-31 21:16:33  129219117 tr_wt.pth
2020-10-31 21:16:33     577263 tr_wt_vocab.pkl
`

`
s3://[golder]-ml/trlanguagemodel/docs/tr_databunch
`

`
(base)  $ aws s3 ls s3://[folder]-ml/trmoviesentiment/models/
2020-10-31 21:34:11  387655273 trfine_tuned.pth
2020-10-31 21:34:11  129098532 trfine_tuned_enc.pth
2020-10-31 21:46:14  170109634 trclas.pth
`

In [19]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [20]:
lr = 1e-3
lr *= bs/48

In [21]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.780294,4.124835,0.306853,00:42


In [22]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.137742,3.990493,0.326406,00:47
1,3.983570,3.876015,0.342746,00:47
2,3.765752,3.800139,0.352879,00:47
3,3.518003,3.777829,0.361429,00:48
4,3.291632,3.797099,0.363013,00:48


In [23]:
learn_lm.save(f'{lang}fine_tuned_prodlambdacontainer')
learn_lm.save_encoder(f'{lang}fine_tuned_enc_prodlambdacontainer')

### Classifier

In [30]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [31]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc_prodlambdacontainer')
learn_c.freeze()

In [32]:
lr=2e-2
lr *= bs/48

In [33]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.463947,0.399002,0.827392,00:19
1,0.424544,0.361432,0.845216,00:17


In [34]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.321796,0.277001,0.889306,00:36
1,0.243321,0.258124,0.895872,00:37


In [35]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.175803,0.270081,0.889306,00:49
1,0.154200,0.273559,0.891182,00:44
2,0.103779,0.308990,0.896811,00:55
3,0.074725,0.285419,0.902439,00:46


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [36]:
learn_c.save(f'{lang}clas_prodlambdacontainer_trsentiment')

In [38]:
learn_c.path

PosixPath('/home/ec2-user/.fastai/data/trwiki/trmoviesentiment')

In [39]:
learn_c.export()

## fin

In [40]:
learn_c.predict("gerçekten harika bir yapim birçok kez izledim")

(Category tensor(1), tensor(1), tensor([9.6786e-04, 9.9903e-01]))

In [42]:
learn_c.predict("olmamış. beğenmedim")

(Category tensor(0), tensor(0), tensor([0.7187, 0.2813]))